### Teorema de Bayes

- Distribució Conjunta
- Distribució Condicional - Versemblança -
- Distribució a priori
- Distribució a posteriori

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max.columns', None)
import seaborn as sns
sns.set_theme()

In [2]:
iris = sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
from scipy.stats import gaussian_kde

#### Variable independent: X

- X és un vector amb els valors dels atributs (columnes): $\vec{X}=\left(x_1, x_2, \dots, x_m\right)$
- per exemple, la observació 0 és el vector: $\vec{X_0}=\left(5.1, 3.5, 1.4, 0.2\right)$ 

#### Variable dependent: Y

- en el cas genèric seria també un vector: $\vec{Y}=\left(y_1, y_2, \dots, y_l\right)$
- en problemes de classificació acostuma a ser una categoria
- en aquest cas: $Y\in\{\text{"setosa", "virginica", "versicolor"}\}$, per exemple: $Y_0=\text{"setosa"}$

### Joint input distribution: $P\left(X\right)$

- és la distribució de la variable independent, un vector amb tots els atributs que tenim en el dataset
- en aquest cas serà una distribució conjunta de 4 dimensions
- la podem estimar a partir de l'histograma conjunt o amb un KDE si la variable és contínua

In [4]:
joint_pdf = gaussian_kde(iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].transpose())

- podem avaluar la densitat per cada un dels exemples

In [5]:
example = iris.iloc[np.random.randint(150)]
species, values = example.species, tuple(example[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
jointpdf = joint_pdf.evaluate(values).item()
print('Example is %s ... density: %6.4f' %(example.species, jointpdf))

Example is versicolor ... density: 0.2115


- però de què ens serveix això ??? ... 

### Conditional distribution: $P\left(X|Y\right)$

In [6]:
setosa = iris.loc[iris.species == 'setosa'][['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

In [7]:
pdf_setosa = gaussian_kde(setosa.transpose(), bw_method = 'scott')

In [8]:
virginica = iris.loc[iris.species == 'virginica'][['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

In [9]:
pdf_virginica = gaussian_kde(virginica.transpose(), bw_method = 'scott')

In [10]:
versicolor = iris.loc[iris.species == 'versicolor'][['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

In [11]:
pdf_versicolor = gaussian_kde(versicolor.transpose(), bw_method = 'scott')

### Likelihood function (versemblança)  $\mathcal{L}\left(\theta, X\right)\approxeq\; P\left(X|Y\right)$

In [12]:
idx = np.random.randint(50)
example = iris.loc[iris.species == 'setosa'].reset_index().iloc[idx]
example

index               22
sepal_length       4.6
sepal_width        3.6
petal_length       1.0
petal_width        0.2
species         setosa
Name: 22, dtype: object

In [13]:
pdf_setosa.evaluate(tuple(example[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]))

array([2.47801801])

In [14]:
pdf_virginica.evaluate(tuple(example[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]))

array([9.26615475e-90])

In [15]:
pdf_versicolor.evaluate(tuple(example[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]))

array([4.96047657e-56])

In [16]:
example = iris.iloc[np.random.randint(150)]
species, values = example.species, tuple(example[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
pdfSetosa = pdf_setosa.evaluate(values).item()
pdfVirginica = pdf_virginica.evaluate(values).item()
pdfVersicolor = pdf_versicolor.evaluate(values).item()
print('Example is %s ... likelihood: setosa %6.4f, virginica %6.4f, versicolor %6.4f' %(example.species, pdfSetosa, pdfVirginica, pdfVersicolor))


Example is versicolor ... likelihood: setosa 0.0000, virginica 0.3739, versicolor 0.8493


- però això són densitats, no són probabilitats !!!!

### Prior distribution: $P\left(Y\right)$

- és la distribució de la variable dependent $Y$
- si no en tenim informació l'estimem a partir de la mostra (igual que la de la variable dependent $X$)
- però podria ser que en tinguessim informació i si fos així la fariem servir (exemple eleccions)

In [17]:
iris.species.value_counts(normalize = True)

setosa        0.333333
versicolor    0.333333
virginica     0.333333
Name: species, dtype: float64

In [18]:
prior = iris.species.value_counts(normalize = True)

### Posterior distribution:  $P\left(Y|X\right)$

- és la distribució condicional de la variable dependent $Y$ donat un valor de la variable independent $X$
- es relaciona amb la distribució condicional $P\left(X|Y\right)$ segons el *Teorema de Bayes*
- el **Teorema de Bayes** quantifica com varia la distribució de la variable dependent 𝑌 deguda al fet d'haver observat una certa evidència (un determinat valor de la variable independent 𝑋)

### <center>Bayes Theorem</center>

$$
\text{if}\;X,\,Y\;\text{dependent variables}
\\
\text{ }
\\
P\left(X, Y\right) = P\left(X|Y\right)\,P\left(Y\right) = P\left(Y|X\right)\,P\left(X\right)
\\
\text{ }
\\
\text{then}
\\
\text{ }
\\
P\left(Y|X\right) = \frac{P\left(Y\right)\,P\left(X|Y\right)}{P\left(X\right)}
$$

### Exemple

- donada una observació d'una Iris qualsevol, **i sense cap altra tipus d'informació adicional**, és a dir sense haver mesurat ni pètals ni sèpals, la nostra "creença" sobre a quína espècie pertany és la que ens dona la *distribució a priori* $P\left(Y\right)$, en aquest cas, d'un 33.3% de probabilitat per cada espècie.
- però si mesurem els seus pètals i sèpals i observem els valors:
    - sepal length: 5.2
    - sepal width: 3.3
    - petal length: 1.5
    - petal width: 0.3
- aleshores, la probabilitat de cada espècie és:

$$
\\
P\left(Y=\text{'setosa'}\,|\,X=\left(5.2, 3.3, 1.5, 0.3\right)\right)=\frac{P\left(Y=\text{'setosa'}\right)\,P\left(X=\left(5.2, 3.3, 1.5, 0.3\right)\,|\,Y=\text{'setosa'}\right)}{P\left(X=\left(5.2, 3.3, 1.5, 0.3\right)\right)}
\\
\text{ }
\\
P\left(Y=\text{'virginica'}\,|\,X=\left(5.2, 3.3, 1.5, 0.3\right)\right)=\frac{P\left(Y=\text{'virginica'}\right)\,P\left(X=\left(5.2, 3.3, 1.5, 0.3\right)\,|\,Y=\text{'virginica'}\right)}{P\left(X=\left(5.2, 3.3, 1.5, 0.3\right)\right)}
\\
\text{ }
\\
P\left(Y=\text{'versicolor'}\,|\,X=\left(5.2, 3.3, 1.5, 0.3\right)\right)=\frac{P\left(Y=\text{'versicolor'}\right)\,P\left(X=\left(5.2, 3.3, 1.5, 0.3\right)\,|\,Y=\text{'versicolor'}\right)}{P\left(X=\left(5.2, 3.3, 1.5, 0.3\right)\right)}
$$

- que substituint per les distribucions que hem estimat de la mostra, dona:

$$
\\
P\left(Y=\text{'setosa'}\,|\,X=\left(5.2, 3.3, 1.5, 0.3\right)\right)=\frac{0.33\;\text{pdf_setosa}\left(5.2, 3.3, 1.5, 0.3\right)}{\text{joint_pdf}\left(5.2, 3.3, 1.5, 0.3\right)}
\\
\text{ }
\\
P\left(Y=\text{'virginica'}\,|\,X=\left(5.2, 3.3, 1.5, 0.3\right)\right)=\frac{0.33\;\text{pdf_virginica}\left(5.2, 3.3, 1.5, 0.3\right)}{\text{joint_pdf}\left(5.2, 3.3, 1.5, 0.3\right)}
\\
\text{ }
\\
P\left(Y=\text{'versicolor'}\,|\,X=\left(5.2, 3.3, 1.5, 0.3\right)\right)=\frac{0.33\;\text{pdf_versicolor}\left(5.2, 3.3, 1.5, 0.3\right)}{\text{joint_pdf}\left(5.2, 3.3, 1.5, 0.3\right)}
$$

### Inferència

- en aquestes expressions, el denominador $P\left(X\right)$ és sempre comú a totes (és només un factor d'escala), per tant podem obviar-lo i considerar:

$$
P\left(Y|X\right) \propto P\left(Y\right)\,P\left(X|Y\right)
$$

- aleshores, podem fer inferència sobre $Y$ mirant per quína classe la probabilitat a posteriori és màxima:

$$
\arg\max_{c\in Y}{P\left(c|X\right) = P\left(c\right)\;P\left(X|c\right)}
$$

- per la mostra que tenim de Iris les tres classes són "a priori" igual de probables (en la realitat no tindria perquè ser així), o sigui:

$$
\forall{c\in Y},\;P\left(c\right) = 0.33
$$

- en general, **sempre que les classes siguin equiprobables**, la probabilitat a posteriori és directament proporcional a la versemblança de les dades:

$$
\forall{c}\in Y,\; P\left(Y=c|X\right) \propto P\left(X|Y=c\right)
$$

- per les dades observades $\vec{X}=\left(5.2, 3.3, 1.5, 0.3\right)$ la classe més versemblant (o més probable) serà:

In [19]:
values = (5.2, 3.3, 1.5, 0.3)
pdfSetosa = pdf_setosa.evaluate(values).item()
pdfVirginica = pdf_virginica.evaluate(values).item()
pdfVersicolor = pdf_versicolor.evaluate(values).item()
print('likelihood: setosa %6.4f, virginica %6.4f, versicolor %6.4f' %(pdfSetosa, pdfVirginica, pdfVersicolor))

likelihood: setosa 3.9084, virginica 0.0000, versicolor 0.0000


- les probabilitats les podem obtenir normalitzant les versemblances obtingudes (perquè les probabilitats de totes les classes han de sumar 1),

In [20]:
sumpdf = pdfSetosa +pdfVirginica +pdfVersicolor
print('posterior probability: setosa %6.4f, virginica %6.4f, versicolor %6.4f' %(pdfSetosa/sumpdf, pdfVirginica/sumpdf, pdfVersicolor/sumpdf))

posterior probability: setosa 1.0000, virginica 0.0000, versicolor 0.0000
